Link: https://www.kaggle.com/competitions/playground-series-s4e8/overview  
Previsão se um cogumelo é venenoso  
Nome: Matheus Amaral e Pedro Gabriel

**BASELINE**

In [ ]:
from google.colab import drive

drive.mount('/gdrive')
%cd /gdrive/MyDrive/rdc/tei

Mounted at /gdrive
/gdrive/MyDrive/rdc/tei


In [ ]:
import pandas as pd

df_test = pd.read_csv("test.csv")
df_train = pd.read_csv("train.csv")

In [ ]:
X_train = df_train.drop(columns=["id", "class"])
y_train = df_train["class"]

**Usando Decision Tree**

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()


for col in X_train.columns:
  if col != 'class':
    le.fit(X_train[col])
    X_train[col] = le.transform(X_train[col])

for col in df_test.columns:
    if col != 'id':
      le.fit(df_test[col])
      df_test[col] = le.transform(df_test[col])

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [ ]:
test_predictions = model.predict(df_test.drop('id', axis=1))

df_test['class'] = test_predictions
df_test[["id","class"]].to_csv("basic-tree.csv", index=False)

Score Kaggle: 0.03818

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=5)
model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5)

In [ ]:
test_predictions = model.predict(df_test.drop('id', axis=1).drop('class', axis=1))

df_test['class'] = test_predictions
df_test[["id","class"]].to_csv("basic-tree2.csv", index=False)

Score Kaggle: 0.25899

**Usando XGBooster**

In [ ]:
import pandas as pd

df_test = pd.read_csv("test.csv")
df_train = pd.read_csv("train.csv")

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()


#transform para o xgbooster porque ele precisa da coluna class em 0 ou 1

for col in df_train.columns:
  le.fit(df_train[col])
  df_train[col] = le.transform(df_train[col])
  if col == 'class':
        le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
        print(le_name_mapping)

for col in df_test.columns:
    if col != 'id':
      le.fit(df_test[col])
      df_test[col] = le.transform(df_test[col])

{'e': 0, 'p': 1}


In [ ]:
X_train = df_train.drop(columns=["id", "class"])
y_train = df_train["class"]

In [ ]:
import xgboost as xgb

model = xgb.XGBClassifier(objective='binary:logistic')
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

Score Kaggle: 0.22827

In [ ]:

test_predictions = model.predict(df_test.drop('id', axis=1))

df_test['class'] = test_predictions
df_test['class'].replace({0: 'e', 1: 'p'}, inplace=True)
df_test[["id","class"]].to_csv("basic-xgb.csv", index=False)